In [1]:
import gspread, requests, os
from oauth2client.service_account import ServiceAccountCredentials
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from dotenv import load_dotenv
import polars as pl

In [2]:
load_dotenv()

True

In [3]:
cv_folder_path = 'data/cv'
study_plan_folder_path = 'data/study_plan'

In [4]:
sheet_position = '2'
file_id = '1i4mB9W2Ef0WjsORA3vadugfXSHiuEJu-i6kj4DNTBsw'

In [14]:
expected_headers = [
    "Added Time",
    "IP Address",
    "St user ID Habacus",
    "I tuoi dati",
    "Email",
    "Hai concluso gli studi post-diploma?",
    "Quando hai conseguito il titolo?",
    "Tra quanto prevedi di concludere gli studi?",
    "Con quale valutazione?",
    "Stai cercando lavoro in questo momento?",
    "Inserisci il tuo URL LinkedIn",
    "Come vuoi procedere?",
    "Carica il tuo libretto universitario",
    "Quando hai iniziato il percorso formativo?",
    "Presso quale ente formativo?",
    "Inserisci il tipo del corso",
    "Inserisci il nome del corso",
    "Come vuoi procedere?",
    "Carica il tuo CV",
    "Non ho esperienza lavorativa",
    "Inizio impiego",
    "Fine impiego",
    "Nome azienda",
    "Nome del ruolo",
    "In cosa consiste?",
    "Hai altre esperienze lavorative che vuoi aggiungere?",
    "Inizio impiego",
    "Fine impiego",
    "Nome Azienda",
    "Nome del ruolo",
    "In cosa consiste?",
    "Hai altre esperienze lavorative che vuoi aggiungere?",
    "Inizio impiego",
    "Fine impiego",
    "Nome Azienda",
    "Nome del ruolo",
    "In cosa consiste?",
    "Non ho esperienze extra-lavorative",
    "Quale/i tra queste attività hai svolto?",
    "Seleziona una lingua",
    "Lingua nativa",
    "Con quale livello?",
    "1_Conosci altre lingue?",
    "Seleziona una lingua",
    "Lingua nativa",
    "Con quale livello?",
    "Conosci altre lingue?",
    "Seleziona una lingua",
    "Lingua nativa",
    "Con quale livello?",
    "Conosci altre lingue?",
    "Seleziona una lingua",
    "Lingua nativa",
    "Con quale livello?",
    "Se potessi scegliere una sfida da affrontare domani al lavoro, quale sarebbe?",
    "In quale di queste attività senti di avere più talento naturale?",
    "Quale frase ti rappresenta meglio quando lavori in gruppo?",
    "Qual è stato il progetto (universitario, personale o professionale) per cui provi più orgoglio?",
    "Immagina che domani un’azienda ti assuma. Qual è il primo aspetto su cui vorresti ricevere formazione?",
    "Hai preferenze su dove lavorare?",
    "Hai mai valutato la possibilità di trasferirti per studiare o lavorare?",
    "Quali località preferiresti?",
    "In che modalità?",
    "Quale modalità di lavoro prediligi?",
    'Zoho CRM',
    'Uuid',	
    "Con Lode",	
    "Terms and Conditions"
]

In [15]:
# Dictionary to keep track of occurrences
header_count = {}

# New list to store modified headers
modified_headers = []

for header in expected_headers:
    if header in header_count:
        header_count[header] += 1
        # Append suffix to the header
        modified_headers.append(f"{header}_{header_count[header]}")
    else:
        header_count[header] = 1
        modified_headers.append(header)


In [16]:
# Define the scope
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]

# Load the service account credentials
creds = ServiceAccountCredentials.from_json_keyfile_name(os.getenv('GOOGLE_APPLICATION_CREDENTIALS'), scope)

# Authorize the client
client = gspread.authorize(creds)

# Open the Google Sheet by name or URL
sheet = client.open('01_Community Hiring').get_worksheet(1)  # or use .get_worksheet(index)


# Get all values from the sheet
data = sheet.get_all_values()[1:]


In [17]:
# Convert to a DataFrame, using the first row as the header
df = pl.DataFrame(data, schema=modified_headers, orient='row')  # Use the first row as the header
df.head()

Added Time,IP Address,St user ID Habacus,I tuoi dati,Email,Hai concluso gli studi post-diploma?,Quando hai conseguito il titolo?,Tra quanto prevedi di concludere gli studi?,Con quale valutazione?,Stai cercando lavoro in questo momento?,Inserisci il tuo URL LinkedIn,Come vuoi procedere?,Carica il tuo libretto universitario,Quando hai iniziato il percorso formativo?,Presso quale ente formativo?,Inserisci il tipo del corso,Inserisci il nome del corso,Come vuoi procedere?_2,Carica il tuo CV,Non ho esperienza lavorativa,Inizio impiego,Fine impiego,Nome azienda,Nome del ruolo,In cosa consiste?,Hai altre esperienze lavorative che vuoi aggiungere?,Inizio impiego_2,Fine impiego_2,Nome Azienda,Nome del ruolo_2,In cosa consiste?_2,Hai altre esperienze lavorative che vuoi aggiungere?_2,Inizio impiego_3,Fine impiego_3,Nome Azienda_2,Nome del ruolo_3,In cosa consiste?_3,Non ho esperienze extra-lavorative,Quale/i tra queste attività hai svolto?,Seleziona una lingua,Lingua nativa,Con quale livello?,1_Conosci altre lingue?,Seleziona una lingua_2,Lingua nativa_2,Con quale livello?_2,Conosci altre lingue?,Seleziona una lingua_3,Lingua nativa_3,Con quale livello?_3,Conosci altre lingue?_2,Seleziona una lingua_4,Lingua nativa_4,Con quale livello?_4,"Se potessi scegliere una sfida da affrontare domani al lavoro, quale sarebbe?",In quale di queste attività senti di avere più talento naturale?,Quale frase ti rappresenta meglio quando lavori in gruppo?,"Qual è stato il progetto (universitario, personale o professionale) per cui provi più orgoglio?",Immagina che domani un’azienda ti assuma. Qual è il primo aspetto su cui vorresti ricevere formazione?,Hai preferenze su dove lavorare?,Hai mai valutato la possibilità di trasferirti per studiare o lavorare?,Quali località preferiresti?,In che modalità?,Quale modalità di lavoro prediligi?,Zoho CRM,Uuid,Con Lode,Terms and Conditions
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""06-Dec-2024 11:37:40""","""""","""""","""Gabriella, Berardi""","""gabriellaberardi.mail@gmail.co…","""No, sto studiando""","""""","""""","""""","""""","""""","""Inserisco solo delle informazi…","""""","""Feb-2022""","""NABA - Nuova Accademia di Bell…","""Laurea triennale""","""DESIGN DEL PRODOTTO""","""Ho un CV""","""https://drive.google.com/file/…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""FAKJFADKFJAFKJAHLFJHWEBFLAJHWE…","""""","""Non ho preferenze""","""Sì""","""Valuterei in base all'opportun…","""""","""""","""""","""""","""""",""""""
"""06-Dec-2024 15:56:20""","""""","""""","""Edoardo, Sezzi""","""edoardo.sezzi@habacus.com""","""Sì""","""Nov-2023""","""""","""100""","""""","""""","""Carico Libretto per trovare la…","""https://drive.google.com/file/…","""""","""""","""""","""""","""Ho un CV""","""https://drive.google.com/file/…","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""Inventare un nuovo prodotto o …","""Ideare soluzioni creative a pr…","""Trovo soluzioni e aiuto il tea…","""Niente in particolare""","""Strumenti o tecnologie specifi…","""Startup, Multinazionali o gran…","""Sì""","""Roma, Estero (Mondo)""","""""","""Ibrida""","""""","""""","""""",""""""


In [18]:
libretti = df.select('Carica il tuo libretto universitario')

In [19]:
for libretto in libretti.rows(named=True):
    print(list(libretto.values())[0])
    lib = list(libretto.values())[0]
    if lib.startswith('https'):
        with open(f'file.pdf', 'wb') as out_file:
            content = requests.get(lib, stream=True).content
            out_file.write(content)


https://drive.google.com/file/d/1Wd6IlZTi7YuEhS1CY14uahzAitP5gmDb/view?usp=drivesdk


In [20]:
def login_with_service_account():
    """
    Google Drive service with a service account.
    note: for the service account to work, you need to share the folder or
    files with the service account email.

    :return: google auth
    """
    # Define the settings dict to use a service account
    # We also can use all options available for the settings dict like
    # oauth_scope,save_credentials,etc.
    settings = {
                "client_config_backend": "service",
                "service_config": {
                    "client_json_file_path": os.getenv('GOOGLE_APPLICATION_CREDENTIALS'),
                }
            }
    # Create instance of GoogleAuth
    gauth = GoogleAuth(settings=settings)
    # Authenticate
    gauth.ServiceAuth()
    return gauth

In [21]:
gauth = login_with_service_account()
drive = GoogleDrive(gauth)

In [22]:
# Define the MIME types for PDF, DOC, and DOCX
mime_types = [
    "application/pdf",
    "application/msword",  # DOC
    "application/vnd.openxmlformats-officedocument.wordprocessingml.document"  # DOCX
]

# Create a query to filter files by MIME type
mime_query = " or ".join([f"mimeType='{mime_type}'" for mime_type in mime_types])


In [23]:
file_list = drive.ListFile({'q': f"trashed=false and ({mime_query})"}).GetList()

In [24]:
for i, file in enumerate(file_list):
  file_title = file['title']
  print('title: %s' % (file_title))
  if file_title.split('_')[0] == 'CV':
    folder_path = cv_folder_path
  elif file_title.split('_')[0] == 'StudyPlan':
    folder_path = study_plan_folder_path
  else:
    print('Prefix is unknown.')
    
  # Create the directory if it doesn't exist
  os.makedirs(folder_path, exist_ok=True)
  file_path = os.path.join(folder_path, f'{file_title}')
  file.GetContentFile(file_path)

title: StudyPlan_EdoardoSezzi.pdf
title: CV_SezziEdoardo.pdf
